# Environment Setup

In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer,*-*"

In [ ]:
#!connect mssql --kernel-name AGDemo-1a "Persist Security Info=False; TrustServerCertificate=True; Integrated Security=true; Initial Catalog=master; Server=rp-sql19ags-1a.perf.rubrik.com;"
#!connect mssql --kernel-name AGDemo-1b "Persist Security Info=False; TrustServerCertificate=True; Integrated Security=true; Initial Catalog=master; Server=rp-sql19ags-1b.perf.rubrik.com;"
#!connect mssql --kernel-name AGDemo-1c "Persist Security Info=False; TrustServerCertificate=True; Integrated Security=true; Initial Catalog=master; Server=rp-sql19ags-1c.perf.rubrik.com;"

In [ ]:
# Rubrik Connection Information
# $ServiceAccountID = op read op://Rubrik/RoadRunnerServiceAccount/username
# $Secret = op read op://Rubrik/RoadRunnerServiceAccount/credential
# $Server = op read op://Rubrik/RoadRunnerServiceAccount/hostname

# Primary SQL Server
$PrimarySQLServerInstance = 'rp-sql19ags-1a.perf.rubrik.com'

# Secondary SQL Servers
$SecondarySQLServerInstance = 'rp-sql19ags-1b.perf.rubrik.com', 'rp-sql19ags-1c.perf.rubrik.com'

# Availability Group Name
$AvailabilityGroupName = 'rp-sql19ags-g1'
# Database Name
$DatabaseName = 'ProductionDatabase'
$TargetDatabaseName = 'ProductionDatabase_Copy'

Import-Module rubrikgqlrunner
$AccessToken = "Bearer"

# Connect to Rubrik

In [ ]:
Connect-RubrikSecurityCloud -AccessToken $AccessToken

## Refresh the Hosts of the SQL Servers in RSC

In [ ]:
$QueryParms = @{
    "hostname" = "$($PrimarySQLServerInstance)"
    "instance" = "MSSQLSERVER"
    "IsRelic" = "false"
}

$RSCMSQLInstance = Invoke-RubrikQuery -Path ./GetMSSQLInstance.gql -QueryParams $QueryParms
$RSCMSQLInstance

$QueryParms = @{
    "id" = "$($RSCMSQLInstance.id)"
}
 Invoke-RubrikQuery -Path ./PhysicalHostRefreshMutation.gql -QueryParams $QueryParms

foreach ($SQLInstance in $SecondarySQLServerInstance){
    $QueryParms = @{
        "hostname" = "$($SQLInstance)"
        "instance" = "MSSQLSERVER"
        "IsRelic" = "false"
    }
    $RSCMSQLInstance = Invoke-RubrikQuery -Path ./GetMSSQLInstance.gql -QueryParams $QueryParms

    $QueryParms = @{
        "id" = "$($RSCMSQLInstance.id)"
    }
    Invoke-RubrikQuery -Path ./PhysicalHostRefreshMutation.gql -QueryParams $QueryParms
}
Start-Sleep -Seconds 45

# Remove Database from Availability Group

In [ ]:
Remove-DbaAgDatabase -SQLInstance $PrimarySQLServerInstance -Database $TargetDatabaseName -Confirm:$false -Verbose

# Remove Database from the SQL Server Instances

In [ ]:
foreach ($SQLInstance in $SecondarySQLServerInstance){
    Remove-DbaDatabase -SqlInstance $SQLInstance -Database $TargetDatabaseName -Confirm:$false -Verbose
}

Remove-DbaDatabase -SqlInstance $PrimarySQLServerInstance -Database $TargetDatabaseName -Confirm:$false -Verbose

## Refresh the Hosts of the SQL Servers in RSC

In [ ]:
$QueryParms = @{
    "hostname" = "$($PrimarySQLServerInstance)"
    "instance" = "MSSQLSERVER"
    "IsRelic" = "false"
}

$RSCMSQLInstance = Invoke-RubrikQuery -Path ./GetMSSQLInstance.gql -QueryParams $QueryParms
$RSCMSQLInstance

$QueryParms = @{
    "id" = "$($RSCMSQLInstance.id)"
}
 Invoke-RubrikQuery -Path ./PhysicalHostRefreshMutation.gql -QueryParams $QueryParms

foreach ($SQLInstance in $SecondarySQLServerInstance){
    $QueryParms = @{
        "hostname" = "$($SQLInstance)"
        "instance" = "MSSQLSERVER"
        "IsRelic" = "false"
    }
    $RSCMSQLInstance = Invoke-RubrikQuery -Path ./GetMSSQLInstance.gql -QueryParams $QueryParms

    $QueryParms = @{
        "id" = "$($RSCMSQLInstance.id)"
    }
    Invoke-RubrikQuery -Path ./PhysicalHostRefreshMutation.gql -QueryParams $QueryParms
}
Start-Sleep -Seconds 45

## Get the ID of the Availability Group

In [ ]:
$QueryParms = @{
    "AvailabilityGroupName" = $($AvailabilityGroupName)
    "IsRelic" = "false"
}
$RSCMSSQLAvailabilityGroup = Invoke-RubrikQuery -Path ./GetMSSQLAvailabilityGroup.gql -QueryParams $QueryParms
$RSCMSSQLAvailabilityGroup

## Get the ID of the Database

In [ ]:
$QueryParms = @{
    "fid" = "$($RSCMSSQLAvailabilityGroup.id)"
    "DatabaseName" = "$($DatabaseName)"
    "IsRelic" = "true"
}
$RSCMSSQLDatabase = Invoke-RubrikQuery -Path ./GetMSSQLDatabaseFromAvailabilityGroup.gql -QueryParams $QueryParms -debug
$RSCMSSQLDatabase.logicalChildConnection.nodes

## Get the Latest Recovery Point

In [ ]:
$QueryParms = @{
    "id" = "$($RSCMSSQLDatabase.logicalChildConnection.nodes[0].id)"
}
$RSCMSSQLRecoverableRange = Invoke-RubrikQuery -Path ./MssqlDatabaseDetailsRecoverableRangesQuery.gql -QueryParams $QueryParms | Sort-Object {$_.data.endTime}
$RSCMSSQLDatabaseLatestRecoveryPoint = ($RSCMSSQLRecoverableRange.data.endTime[-1]).ToUniversalTime().ToString('yyyy-MM-ddTHH:mm:ss.fffZ')
$RSCMSSQLDatabaseLatestRecoveryPoint

# RESTORE FILELIST ONLY from Recovery Point

In [ ]:
$QueryParms = @{
    "input" = @{
        "id" = "$($RSCMSSQLDatabase.logicalChildConnection.nodes[0].id)"
        "time" = "$($RSCMSSQLDatabaseLatestRecoveryPoint)"
    }
}
$RSCMSSQLDatabaseFiles = Invoke-RubrikQuery -Path ./AllMssqlDatabaseRestoreFilesQuery.gql -QueryParams $QueryParms
$RSCMSSQLDatabaseFiles.items
$targetFilePaths = @(
    @{
        logicalName = "$($RSCMSSQLDatabaseFiles.items[0].logicalName)"
        newFilename = "$($TargetDatabaseName).mdf"
        exportPath = "$($RSCMSSQLDatabaseFiles.items[0].originalPath)"
    }
    @{
        logicalName = "$($RSCMSSQLDatabaseFiles.items[1].logicalName)"
        newFilename = "$($TargetDatabaseName).ldf"
        exportPath = "$($RSCMSSQLDatabaseFiles.items[1].originalPath)"
    }
)
$targetFilePaths

## Export the Database to the Secondary Replicas

In [ ]:
foreach ($SQLInstance in $SecondarySQLServerInstance){
    $QueryParms = @{
        "hostname" = "$($SQLInstance)"
        "instance" = "MSSQLSERVER"
        "IsRelic" = "false"
    }

    $RSCMSQLInstance = Invoke-RubrikQuery -Path ./GetMSSQLInstance.gql -QueryParams $QueryParms
    $RSCMSQLInstance    

    $QueryParms = @{
        "input" = @{
            "id" = "$($RSCMSSQLDatabase.logicalChildConnection.nodes[0].id)"
            "config" = @{
                "recoveryPoint" = @{"date" = "$($RSCMSSQLDatabaseLatestRecoveryPoint)"}
                "targetInstanceId" = "$($RSCMSQLInstance.physicalChildConnection.nodes.id)"
                "targetDatabaseName" = "$($TargetDatabaseName)"
                # "targetDataFilePath" = "c:\mnt\sqldata"
                # "targetLogFilePath" = "c:\mnt\sqllogs"
                "targetFilePaths" = $targetFilePaths
                "allowOverwrite" = $true
                "finishRecovery" = $false
            }
        }
    }
    Invoke-RubrikQuery -Path ./MssqlDatabaseExportMutation.gql -QueryParams $QueryParms
}
# Start-Sleep -Seconds 45

# Export the Database to the Primary Replica

In [ ]:
$QueryParms = @{
    "hostname" = "$($PrimarySQLServerInstance)"
    "instance" = "MSSQLSERVER"
    "IsRelic" = "false"
}
$RSCMSQLInstance = Invoke-RubrikQuery -Path ./GetMSSQLInstance.gql -QueryParams $QueryParms
$RSCMSQLInstance    

$QueryParms = @{
    "input" = @{
        "id" = "$($RSCMSSQLDatabase.logicalChildConnection.nodes[0].id)"
        "config" = @{
            "recoveryPoint" = @{"date" = "$($RSCMSSQLDatabaseLatestRecoveryPoint)"}
            "targetInstanceId" = "$($RSCMSQLInstance.physicalChildConnection.nodes.id)"
            "targetDatabaseName" = "$($TargetDatabaseName)"
            # "targetDataFilePath" = "c:\mnt\sqldata"
            # "targetLogFilePath" = "c:\mnt\sqllogs"
            "targetFilePaths" = $targetFilePaths
            "allowOverwrite" = $true
            "finishRecovery" = $true
        }
    }
}
Invoke-RubrikQuery -Path ./MssqlDatabaseExportMutation.gql -QueryParams $QueryParms
Start-Sleep -Seconds 45

# Add database into availability group on primary replica

In [ ]:
$Query = "ALTER AVAILABILITY GROUP [$($AvailabilityGroupName)] ADD DATABASE [$($TargetDatabaseName)];"
Invoke-DbaQuery -SqlInstance $PrimarySQLServerInstance -Query $Query

In [ ]:
foreach ($SQLInstance in $SecondarySQLServerInstance){
    $Query = "ALTER DATABASE [$($DatabaseName)] SET HADR AVAILABILITY GROUP = [$($AvailabilityGroupName)];"
    Invoke-DbaQuery -SqlInstance $SQLInstance -Query $Query
}